In [ ]:
#!/usr/bin/env python

from fanuc_pcdk_client import FanucPCDKClient
from robot_loc_world import RobotLocWorld
from robot_loc_joint import RobotLocJoint
from time import sleep
from copy import deepcopy
import sys

ROBOT_SPEED = 30
ROBOT_SPEED_LINEAR = 45

In [ ]:
import sys

In [ ]:
# Robot init
robot = FanucPCDKClient()
robot.connect()

# Run teach pendant program.
robot.run_program('PCDK')
robot.reset_alarms()

# Adjust the speed of the robot
robot.set_speed(ROBOT_SPEED)

==== right
x=402.368, y=362.836, z=-302.223, w=-177.564, p=2.676, r=-132.382, f=False, u=True, t=True

x=401.985, y=366.375, z=-295.562, w=179.699, p=0.962, r=47.409, f=False, u=True, t=True

==== left
x=425.330, y=-356.436, z=-180.083, w=-179.407, p=-4.169, r=-53.070, f=False, u=True, t=True

In [ ]:
loc_world = robot.get_position("world")
print(f'Current world location: {loc_world}\n')

In [ ]:
world_loc = RobotLocWorld(x=401.985, y=366.375, z=-155.562, w=179.699, p=0.962, r=47.409, f=False, u=True, t=True)
robot.move(world_loc, move_linear=True, speed=ROBOT_SPEED_LINEAR)

In [ ]:
# Going to need to manually get the different poses
# Maybe need 4 or 5 waypoints

### NEEDS TO BE CALIBRATED
peg1_start = RobotLocWorld(x=345.437, y=128.544, z=-119.671,
                          w=179.499, p=0.827, r=2.541, 
                          f=False, u=True, t=True)

peg1_end = RobotLocWorld(x=345.437, y=128.544, z=-119.671,
                          w=179.499, p=0.827, r=2.541, 
                          f=False, u=True, t=True)

peg2_start = RobotLocWorld(x=345.437, y=128.544, z=-119.671,
                          w=179.499, p=0.827, r=2.541, 
                          f=False, u=True, t=True)

peg2_end = RobotLocWorld(x=345.437, y=128.544, z=-119.671,
                          w=179.499, p=0.827, r=2.541, 
                          f=False, u=True, t=True)

peg3_start = RobotLocWorld(x=345.437, y=128.544, z=-119.671,
                          w=179.499, p=0.827, r=2.541, 
                          f=False, u=True, t=True)

peg3_end = RobotLocWorld(x=345.437, y=128.544, z=-119.671,
                          w=179.499, p=0.827, r=2.541, 
                          f=False, u=True, t=True)

In [ ]:
# 
peg1_traj = [home_pose, above_start, peg1_start, 'close', above_peg, home_pose, above_end, peg1_end, 'open', above_end, home_pose]
peg2_traj = [home_pose, above_start, peg2_start, 'close', above_peg, home_pose, above_end, peg2_end, 'open', above_end, home_pose]
peg3_traj = [home_pose, above_start, peg3_start, 'close', above_peg, home_pose, above_end, peg3_end, 'open', above_end, home_pose]

trajectories = {1: [peg1_traj], 2: [peg2_traj, peg1_traj], 3: [peg3_traj, peg2_traj, peg1_traj]}

In [22]:
def execute_insertion(pegs):
    print("\n\n===============    STARTING PEG STACKING    ===============")
    success = []
    
    for traj in trajectories[pegs]:
        
        for pose in traj:
            pegs_moved = len(trajectories[pegs]) - len(success)
            time.sleep(0.5)
            if(pose == 'close'):
                robot.close_gripper()
                print("Picking up the peg #" + str(pegs_moved + 1) + " now!")
            elif(pose == 'open'):
                robot.open_gripper()
                success.append(traj)
                print(" peg #" + str(pegs_moved + 1) + " has been completed!")
            else:
                robot.move(pose, move_linear=True, speed=ROBOT_SPEED_LINEAR)
    
    failed = [traj for traj in trajectories[pegs] if traj not in success]
    
    print("\n\n============================\n      STACKING SUMMARY\n============================")
    print("Successfully stacked " + str(len(success)) + " pegs")
    print("Failed to stack " + str(len(failed)) + " pegs")

In [24]:
terminate = False
user_input = 0
started = False
while not terminate:
    try:
        if(not started):
            print("===============  WELCOME TO PEG STACKING  ===============")
            start = True
            
        user_input = (raw_input('How many pegs are there to move? (press q to quit)\n> '))
        user_input = (user_input) if user_input == 'q' else int(user_input)
        if(user_input == 'q'):
            print("===============    TERMINATING PEG STACKING    ===============")
            terminate = True
        elif(user_input > 0):
            
            '''
            ACTUAL PEG STACK STUFF GOES HERE
            '''
            execute_insertion(user_input)
            
        else:
            print("Invalid choice, please try again.")
            terminate = True
        
        if(available_pegs < 1):
            terminate = True
            
    except ValueError:
        print("Not a valid number, please try again")

===============  WELCOME TO PEG STACKING  ===============
How many pegs are there to move? (press q to quit)
> q
===============    TERMINATING PEG STACKING    ===============
